In [ ]:
import copy
import math
import random
from matplotlib import pyplot as plt
import networkx as nx

class Simulated_Annealing:
    def __init__(self):
        self.Initial_Cycle = []
        self.Initial_Distance = 0
        self.Optimal_Cycle = []
        self.Optimal_Distance = 0
        self.Matrix = None
        self.Number = None
        self.Maximum_Temperature = 1000
        self.Cooling_Rate = 0.99995
        self.Minimum_Temperature = 0.00001
        self.Log = []

    def Import_Matrix(self):
        with open("100_Cities.txt",'r') as File:
            Lines = File.readlines()
            self.Number = int(Lines.pop(0))
            self.Matrix = [[0 for Cols in range(self.Number)] for Rows in range(self.Number)]
            for Row in range(self.Number):
                Line = Lines[Row].split()
                if len(Line) == self.Number:
                    for Col in range(len(Line)):
                        self.Matrix[Row][Col] = float(Line[Col])
                else:
                    print("Đây không phải ma trận vuông")
                    exit(0)
            self.Create_Initial_Cycle()

    def Export_Result(self):
        with open("Result.txt",'w') as File:
            File.write("Kết quả mô phỏng luyện kim\n\n".upper())
            File.write(f"{'':<30}{'Khoảng cách:':<30}{'Chu trình:'}\n")
            File.write(f"{'Chu trình ban đầu:':<30}{self.Initial_Distance:<30}{self.Initial_Cycle}\n")
            File.write(f"{'Chu trình tối ưu:':<30}{self.Optimal_Distance:<30}{self.Optimal_Cycle}\n")
            File.write(f"Ngắn hơn {self.Initial_Distance-self.Optimal_Distance} so với lúc ban đầu\n")
            File.write("Danh sách cung:\n")
            File.write(f"{'':<30}{'Chu trình Ban đầu:':<30}{'Chu trình tối ưu:':<30}\n")
            for i in range(self.Number-1):
                uf = self.Initial_Cycle[i]
                vf = self.Initial_Cycle[i+1]
                wf = self.Matrix[uf][vf]
                uo = self.Optimal_Cycle[i]
                vo = self.Optimal_Cycle[i+1]
                wo = self.Matrix[uo][vo]
                Linef = f"{uf} <-> {vf} = {wf}"
                Lineo = f"{uo} <-> {vo} = {wo}"
                File.write(f"{'':<30}{Linef:<30}{Lineo:<30}\n")
            uf = self.Initial_Cycle[-1]
            vf = self.Initial_Cycle[0]
            wf = self.Matrix[uf][vf]
            uo = self.Optimal_Cycle[-1]
            vo = self.Optimal_Cycle[0]
            wo = self.Matrix[uo][vo]
            Linef = f"{uf} <-> {vf} = {wf}"
            Lineo = f"{uo} <-> {vo} = {wo}"
            File.write(f"{'':<30}{Linef:<30}{Lineo:<30}\n")

    def Export_Log(self):
        File = open("Log.txt", "w")
        File.write("Quá trình mô phỏng luyện kim\n\n".upper())
        File.write(f"{'Nhiệt độ:':<30}{'Khoảng cách:':<30}{'Chu trình:'}\n")
        for i in range(len(self.Log)):
            File.write(f"{self.Log[i][0]:<30}{self.Log[i][1]:<30}{self.Log[i][2]}\n")
        File.close()

    def Nearest_Neighbor(self):
        Unvisited_Cities = list(range(self.Number))
        Current_City = random.choice(Unvisited_Cities)
        Cycle = [Current_City]
        Unvisited_Cities.remove(Current_City)
        while Unvisited_Cities:
            Nearest_City = min(Unvisited_Cities, key=lambda City: self.Matrix[Current_City][City])
            Cycle.append(Nearest_City)
            Current_City = Nearest_City
            Unvisited_Cities.remove(Current_City)
        return Cycle

    def Create_Initial_Cycle(self):
        self.Initial_Cycle = self.Nearest_Neighbor()
        self.Initial_Distance = self.Cycle_Distance(self.Initial_Cycle)

    def Convert_To_Matrix(self, Array):
        Cycle_Matrix = [[0 for Cols in range(self.Number)] for Rows in range(self.Number)]
        for i in range(self.Number-1):
            Cycle_Matrix[Array[i]][Array[i+1]] = self.Matrix[Array[i]][Array[i+1]]
        Cycle_Matrix[Array[-1]][Array[0]] = self.Matrix[Array[-1]][Array[0]]
        return Cycle_Matrix

    def Cycle_Distance(self, Array):
        Distance = 0
        for i in range(self.Number-1):
            Distance += self.Matrix[Array[i]][Array[i+1]]
        Distance += self.Matrix[Array[-1]][Array[0]]
        return Distance

    def Create_Random_Cycle(self, Array):
        New_Array = copy.deepcopy(Array)
        First, End = sorted(random.sample(range(self.Number), 2))
        New_Array[First:End+1] = reversed(New_Array[First:End+1])
        return New_Array

    def Acceptance(self, Delta, Current_Temperature):
        return math.exp(-Delta/Current_Temperature)

    def Annealing(self):
        self.Cycle_Graph_TSP(self.Initial_Cycle,"Chu trình ban đầu")
        Current_Cycle = copy.deepcopy(self.Initial_Cycle)
        Current_Distance = self.Initial_Distance
        Current_Temperature = self.Maximum_Temperature
        Distances = []
        Distances.append(self.Initial_Distance)
        self.Log.append([self.Maximum_Temperature, self.Initial_Distance, self.Initial_Cycle])
        while Current_Temperature >= self.Minimum_Temperature:
            Candidate_Cycle = self.Create_Random_Cycle(Current_Cycle)
            Candidate_Distance = self.Cycle_Distance(Candidate_Cycle)
            Delta = Candidate_Distance - Current_Distance
            if Delta < 0 or self.Acceptance(Delta, Current_Temperature) > random.random():
                Current_Cycle = copy.deepcopy(Candidate_Cycle)
                Current_Distance = Candidate_Distance
            Current_Temperature *= self.Cooling_Rate
            Distances.append(Current_Distance)
            self.Log.append([Current_Temperature, Current_Distance, Current_Cycle])
        self.Optimal_Cycle = copy.deepcopy(Current_Cycle)
        self.Optimal_Distance = Current_Distance
        self.Line_Graph_SA(list(range(len(Distances))), Distances)
        self.Cycle_Graph_TSP(self.Optimal_Cycle, "Chu trình tối ưu")

    def Line_Graph_SA(self, Iteration, Distances):
        plt.figure(figsize=(15,10))
        plt.plot(Iteration, Distances)
        Initial_Line = plt.axhline(y=self.Initial_Distance, color="purple")
        Optimal_Line = plt.axhline(y=self.Optimal_Distance, color="green")
        Initial_Content = f"Khoảng cách ban đầu: {round(self.Initial_Distance,2)}"
        Optimal_Content = f"Khoảng cách tối ưu: {round(self.Optimal_Distance,2)}"
        plt.legend([Initial_Line, Optimal_Line],[Initial_Content, Optimal_Content])
        plt.title("Quá trình mô phỏng luyện kim")
        plt.xlabel("Số lần lặp")
        plt.ylabel("Khoảng cách chu trình(KM)")
        plt.show()

    def Cycle_Graph_TSP(self, Array, Title):
        New_Matrix = self.Convert_To_Matrix(Array)
        New_Graph = nx.Graph()
        plt.figure(figsize=(15,15))
        plt.title(Title, fontsize=20, fontweight="bold")
        for i in range(self.Number):
            for j in range(self.Number):
                if New_Matrix[Array[i]][Array[j]] != 0:
                    New_Graph.add_edge(Array[i], Array[j], weight=New_Matrix[Array[i]][Array[j]])
        Layout = nx.layout.fruchterman_reingold_layout(New_Graph)
        nx.draw(New_Graph, Layout, with_labels=True, node_size=200, node_color='yellow', font_size=10)
        plt.show()

    def Execution(self):
        self.Import_Matrix()
        self.Annealing()
        #self.Export_Result()
        #self.Export_Log()

SA = Simulated_Annealing()
SA.Execution()